In [1]:
import sys
sys.path.append("pytorch")
from pytorch.main import *
from pytorch.core import *
from pathlib import Path
import csv

In [2]:
DATASET_DIR =Path('/media/Extra_Drive/fushigi_nn_data')
DATASET_NAM ='13Feb2019'
DATASET_PATH=DATASET_DIR/(DATASET_NAM+'.h5')
RESULTS_PATH=DATASET_DIR/(DATASET_NAM+'.out.tsv')
LABELS_PATH='class_labels_indices.csv'

MODELS_PATH=Path('/media/Extra_Drive/audioset_classification/models/main/balance_type=balance_in_batch/model_type=decision_level_single_attention')
MODEL_FNAME='md_50000_iters.tar'
MODEL_PATH=MODELS_PATH/MODEL_FNAME

In [3]:
(x, y, id_list) = utilities.load_data(DATASET_PATH)

In [4]:
xt, yt = utilities.transform_data(x, y)

In [5]:
freq_bins = 128
classes_num = 527

# Hyper parameters
hidden_units = 1024
drop_rate = 0.0
batch_size = 1

In [6]:
model = DecisionLevelSingleAttention(freq_bins, classes_num, hidden_units, drop_rate)

In [7]:
model.load_state_dict(torch.load(str(MODEL_PATH))['state_dict'])

In [8]:
model.cuda()

DecisionLevelSingleAttention(
  (emb): EmbeddingLayers(
    (conv1): Conv2d(128, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv2): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv3): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (bn1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True)
    (bn2): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True)
    (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True)
  )
  (attention): Attention(
    (att): Conv2d(1024, 527, kernel_size=(1, 1), stride=(1, 1))
    (cla): Conv2d(1024, 527, kernel_size=(1, 1), stride=(1, 1))
  )
)

In [9]:
model.eval()

DecisionLevelSingleAttention(
  (emb): EmbeddingLayers(
    (conv1): Conv2d(128, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv2): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv3): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (bn1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True)
    (bn2): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True)
    (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True)
  )
  (attention): Attention(
    (att): Conv2d(1024, 527, kernel_size=(1, 1), stride=(1, 1))
    (cla): Conv2d(1024, 527, kernel_size=(1, 1), stride=(1, 1))
  )
)

In [10]:
batch_x = move_data_to_gpu(xt, True, volatile=True)

In [11]:
output = model(batch_x)

In [12]:
len(output)

6212

In [13]:
# get idx to label map
import csv
labels = []
with open(LABELS_PATH) as inf:
    rd = csv.reader(inf)
    for i, row in enumerate(rd):
        if i == 0: continue
        labels.append(row[-1].strip())
assert labels[0] == 'Speech'

In [14]:
id_list[:10]

[b'dane_recording_07_32_50_20190211.pcm_out00000.wav',
 b'dane_recording_07_32_50_20190211.pcm_out00001.wav',
 b'dane_recording_07_32_50_20190211.pcm_out00002.wav',
 b'dane_recording_07_32_50_20190211.pcm_out00003.wav',
 b'dane_recording_07_32_50_20190211.pcm_out00004.wav',
 b'dane_recording_07_32_50_20190211.pcm_out00005.wav',
 b'dane_recording_07_32_50_20190211.pcm_out00006.wav',
 b'dane_recording_07_32_50_20190211.pcm_out00007.wav',
 b'dane_recording_07_32_50_20190211.pcm_out00008.wav',
 b'dane_recording_07_32_50_20190211.pcm_out00009.wav']

In [24]:
with open(RESULTS_PATH, 'w') as out:
    for j, idd in enumerate(id_list):
        top_label_pairs = sorted(
            [(labels[i], output[j][i].data.cpu().numpy()[0]) for i in range(len(output[j]))], 
            reverse=True,
            key=lambda x: x[1]
        )[:5]
        print(str(id_list[j]), *[item for pair in top_label_pairs for item in pair], sep='\t', file=out)

In [13]:

for j in range(len(output)):
    print(id_list[j], 
          sorted(
              [
                  (output[j][i].data.cpu().numpy()[0], i) for i in range(len(output[1]))
              ], reverse=True)[:5]
         )

b'dane_recording_07_32_50_20190211.pcm_out00042.wav' [(0.03152176, 500), (0.030619426, 137), (0.012039031, 72), (0.008490477, 124), (0.0048019523, 0)]
b'dane_recording_07_32_50_20190211.pcm_out00049.wav' [(0.021255016, 137), (0.019773092, 500), (0.011538644, 72), (0.008412017, 124), (0.0070929183, 407)]
b'dane_recording_07_32_50_20190211.pcm_out00041.wav' [(0.032303825, 500), (0.026764164, 137), (0.012453003, 72), (0.01053946, 124), (0.005814719, 0)]
b'dane_recording_07_32_50_20190211.pcm_out00063.wav' [(0.06428144, 500), (0.018247897, 137), (0.016936587, 72), (0.011519176, 124), (0.006978593, 300)]
b'dane_recording_07_32_50_20190211.pcm_out00020.wav' [(0.043736026, 500), (0.021349631, 137), (0.012408363, 72), (0.009191033, 124), (0.0059403945, 515)]
b'dane_recording_07_32_50_20190211.pcm_out00057.wav' [(0.019636167, 137), (0.01380678, 500), (0.012526613, 72), (0.009197997, 124), (0.0057573696, 0)]
b'dane_recording_07_32_50_20190211.pcm_out00032.wav' [(0.9821172, 384), (0.93897396, 386

b'dane_recording_07_32_50_20190211.pcm_out00028.wav' [(0.9835007, 384), (0.9516652, 386), (0.3235635, 0), (0.03850294, 137), (0.0102800885, 385)]
b'dane_recording_07_38_18_20190211.pcm_out00042.wav' [(0.03100342, 137), (0.011176756, 500), (0.007948915, 72), (0.007320625, 0), (0.004526617, 300)]
b'dane_recording_07_38_18_20190211.pcm_out00049.wav' [(0.023767317, 137), (0.018877266, 500), (0.009460095, 72), (0.0080230795, 0), (0.0065608425, 300)]
b'dane_recording_07_38_18_20190211.pcm_out00041.wav' [(0.032690626, 500), (0.02098412, 137), (0.011027223, 72), (0.0076082805, 124), (0.007215454, 0)]
b'dane_recording_07_38_18_20190211.pcm_out00063.wav' [(0.024585476, 137), (0.012531051, 500), (0.010299892, 72), (0.0065266504, 0), (0.006094649, 300)]
b'dane_recording_07_38_18_20190211.pcm_out00020.wav' [(0.037186097, 407), (0.030563919, 408), (0.018126829, 137), (0.01529013, 404), (0.010496055, 506)]
b'dane_recording_07_38_18_20190211.pcm_out00057.wav' [(0.025546758, 137), (0.013458357, 500), (

b'dane_recording_07_38_18_20190211.pcm_out00030.wav' [(0.98736346, 384), (0.9528613, 386), (0.07461532, 0), (0.018164929, 385), (0.015004009, 137)]
b'dane_recording_07_38_18_20190211.pcm_out00028.wav' [(0.9853317, 384), (0.9485647, 386), (0.060694307, 0), (0.051812954, 137), (0.013244323, 385)]
b'dane_recording_07_43_46_20190211.pcm_out00042.wav' [(0.020552972, 500), (0.019542063, 137), (0.013611897, 72), (0.007118378, 124), (0.006215998, 300)]
b'dane_recording_07_43_46_20190211.pcm_out00049.wav' [(0.021133246, 137), (0.020242987, 500), (0.016329888, 72), (0.008890867, 124), (0.007782001, 0)]
b'dane_recording_07_43_46_20190211.pcm_out00041.wav' [(0.02188782, 137), (0.015233526, 72), (0.013626402, 500), (0.007624807, 0), (0.006863227, 124)]
b'dane_recording_07_43_46_20190211.pcm_out00063.wav' [(0.034525577, 500), (0.015444852, 72), (0.014860038, 137), (0.011311088, 124), (0.0100444015, 57)]
b'dane_recording_07_43_46_20190211.pcm_out00020.wav' [(0.02304533, 137), (0.017772458, 500), (0.0

b'dane_recording_07_43_46_20190211.pcm_out00045.wav' [(0.037875053, 500), (0.02189643, 137), (0.012926098, 72), (0.009160014, 124), (0.0071923104, 300)]
b'dane_recording_07_43_46_20190211.pcm_out00030.wav' [(0.033604942, 500), (0.020285267, 137), (0.013958687, 72), (0.009918247, 124), (0.006032015, 300)]
b'dane_recording_07_43_46_20190211.pcm_out00028.wav' [(0.032647707, 137), (0.009757311, 0), (0.009449341, 72), (0.0044572577, 500), (0.0038756675, 96)]
b'dane_recording_07_49_14_20190211.pcm_out00042.wav' [(0.026853288, 137), (0.015242302, 500), (0.008722861, 0), (0.0068063997, 72), (0.004988284, 124)]
b'dane_recording_07_49_14_20190211.pcm_out00049.wav' [(0.15419598, 57), (0.13298728, 60), (0.105638005, 474), (0.057160422, 518), (0.052445076, 96)]
b'dane_recording_07_49_14_20190211.pcm_out00041.wav' [(0.02379128, 137), (0.018862255, 500), (0.012084824, 72), (0.007246502, 124), (0.005473088, 0)]
b'dane_recording_07_49_14_20190211.pcm_out00063.wav' [(0.016577331, 500), (0.015655346, 72)

b'dane_recording_07_49_14_20190211.pcm_out00006.wav' [(0.031871606, 500), (0.020732284, 137), (0.01337521, 72), (0.010263651, 124), (0.0064311814, 300)]
b'dane_recording_07_49_14_20190211.pcm_out00045.wav' [(0.062968284, 500), (0.019352937, 72), (0.016167387, 124), (0.014811008, 137), (0.0064983685, 506)]
b'dane_recording_07_49_14_20190211.pcm_out00030.wav' [(0.022801556, 500), (0.02037463, 137), (0.014840432, 72), (0.009069817, 124), (0.005741765, 0)]
b'dane_recording_07_49_14_20190211.pcm_out00028.wav' [(0.043851722, 500), (0.01867733, 137), (0.014167085, 72), (0.012284337, 124), (0.0064846342, 300)]
b'dane_recording_07_54_42_20190211.pcm_out00042.wav' [(0.028539347, 137), (0.023200054, 500), (0.008499544, 0), (0.008341217, 72), (0.005708834, 124)]
b'dane_recording_07_54_42_20190211.pcm_out00049.wav' [(0.0445834, 500), (0.026804376, 137), (0.013465375, 72), (0.011991687, 124), (0.005703915, 0)]
b'dane_recording_07_54_42_20190211.pcm_out00041.wav' [(0.03020403, 137), (0.019700177, 500

...

b'dane_recording_19_20_46_20190208.pcm_out00193.wav' [(0.46951866, 72), (0.39020416, 0), (0.28646418, 87), (0.25136206, 430), (0.23385713, 88)]
b'dane_recording_19_20_46_20190208.pcm_out00182.wav' [(0.5421189, 429), (0.49270228, 427), (0.45279783, 428), (0.42628494, 0), (0.32953554, 430)]
b'dane_recording_19_20_46_20190208.pcm_out00099.wav' [(0.5228763, 0), (0.34209996, 137), (0.06031071, 32), (0.051722687, 300), (0.03718998, 307)]
b'dane_recording_19_20_46_20190208.pcm_out00116.wav' [(0.5403272, 0), (0.36407894, 88), (0.29467428, 87), (0.28329796, 430), (0.24826175, 72)]
b'dane_recording_19_20_46_20190208.pcm_out00141.wav' [(0.42086488, 72), (0.40375423, 87), (0.40304464, 88), (0.35837823, 89), (0.2886967, 0)]
b'dane_recording_19_20_46_20190208.pcm_out00188.wav' [(0.49649477, 427), (0.4437127, 0), (0.4400437, 430), (0.22265218, 428), (0.1722652, 98)]
b'dane_recording_19_20_46_20190208.pcm_out00160.wav' [(0.42983133, 0), (0.3899125, 427), (0.3360807, 430), (0.17900936, 428), (0.1498778

b'dane_recording_19_42_37_20190208.pcm_out00208.wav' [(0.26163673, 0), (0.03049272, 137), (0.006218626, 72), (0.0023848508, 506), (0.0021301096, 524)]
b'dane_recording_19_42_37_20190208.pcm_out00253.wav' [(0.6492425, 137), (0.07835311, 0), (0.027990341, 524), (0.002952834, 72), (0.0021815263, 407)]
b'dane_recording_19_42_37_20190208.pcm_out00138.wav' [(0.6091598, 0), (0.107196994, 73), (0.10649642, 72), (0.09269571, 81), (0.05908776, 83)]
b'dane_recording_19_42_37_20190208.pcm_out00213.wav' [(0.76889724, 137), (0.11680988, 0), (0.03619964, 32), (0.031226255, 34), (0.030552814, 37)]
b'dane_recording_19_42_37_20190208.pcm_out00168.wav' [(0.56069076, 0), (0.15761234, 137), (0.144165, 72), (0.11684243, 98), (0.10023747, 99)]
b'dane_recording_19_42_37_20190208.pcm_out00158.wav' [(0.028170539, 0), (0.022902962, 137), (0.007883732, 72), (0.0071955672, 500), (0.003451989, 407)]
b'dane_recording_19_42_37_20190208.pcm_out00088.wav' [(0.4134291, 0), (0.078478366, 72), (0.035421927, 73), (0.023989

b'dane_recording_19_42_37_20190208.pcm_out00244.wav' [(0.061796747, 0), (0.020173604, 137), (0.0093295695, 72), (0.0049312436, 300), (0.003684308, 124)]
b'dane_recording_19_42_37_20190208.pcm_out00091.wav' [(0.64389884, 0), (0.18875831, 5), (0.052451055, 72), (0.04945548, 96), (0.042961095, 97)]
b'dane_recording_19_42_37_20190208.pcm_out00143.wav' [(0.050444458, 0), (0.02526086, 137), (0.013829883, 72), (0.009647893, 500), (0.007680175, 124)]
b'dane_recording_19_42_37_20190208.pcm_out00124.wav' [(0.35585946, 0), (0.15592347, 72), (0.09983539, 73), (0.071603045, 81), (0.041392226, 111)]
b'dane_recording_19_42_37_20190208.pcm_out00151.wav' [(0.6995523, 0), (0.11530885, 5), (0.10345807, 1), (0.04172112, 137), (0.039034195, 46)]
b'dane_recording_19_42_37_20190208.pcm_out00161.wav' [(0.036559083, 0), (0.024252437, 137), (0.014448633, 506), (0.008733783, 354), (0.0068819784, 52)]
b'dane_recording_19_42_37_20190208.pcm_out00062.wav' [(0.5203742, 0), (0.042157184, 137), (0.040509943, 72), (0.0

b'dane_recording_19_42_37_20190208.pcm_out00207.wav' [(0.4901452, 0), (0.0978424, 72), (0.04483654, 90), (0.042779434, 86), (0.034360457, 91)]
b'dane_recording_19_42_37_20190208.pcm_out00018.wav' [(0.5512023, 137), (0.21976085, 0), (0.032264687, 524), (0.030934291, 32), (0.029534569, 525)]
b'dane_recording_19_42_37_20190208.pcm_out00206.wav' [(0.1421082, 0), (0.015663143, 137), (0.012850754, 46), (0.0075449343, 72), (0.0059323963, 506)]
b'dane_recording_19_42_37_20190208.pcm_out00050.wav' [(0.88815415, 137), (0.07961918, 524), (0.028341364, 52), (0.025341313, 0), (0.011541234, 88)]
b'dane_recording_19_42_37_20190208.pcm_out00216.wav' [(0.32542166, 137), (0.11766678, 0), (0.062376704, 126), (0.05913688, 129), (0.05864669, 72)]
b'dane_recording_19_42_37_20190208.pcm_out00238.wav' [(0.21055591, 0), (0.0348144, 137), (0.0061798133, 5), (0.004486078, 515), (0.004146666, 506)]
b'dane_recording_19_42_37_20190208.pcm_out00007.wav' [(0.46917564, 137), (0.2350733, 0), (0.10425192, 96), (0.084911

b'dane_recording_19_42_37_20190208.pcm_out00090.wav' [(0.38648328, 0), (0.057676088, 72), (0.05541758, 90), (0.04755819, 86), (0.044791132, 91)]
b'dane_recording_19_42_37_20190208.pcm_out00118.wav' [(0.32514542, 0), (0.02789889, 137), (0.026638217, 72), (0.015646864, 111), (0.011875633, 115)]
b'dane_recording_19_42_37_20190208.pcm_out00193.wav' [(0.5363192, 0), (0.09061069, 72), (0.07807112, 81), (0.06037635, 73), (0.05755256, 83)]
b'dane_recording_19_42_37_20190208.pcm_out00182.wav' [(0.6421199, 0), (0.031158024, 72), (0.029506344, 5), (0.0181828, 137), (0.01691421, 81)]
b'dane_recording_19_42_37_20190208.pcm_out00099.wav' [(0.65636927, 0), (0.081300646, 111), (0.08046161, 72), (0.049267396, 115), (0.044033993, 73)]
b'dane_recording_19_42_37_20190208.pcm_out00116.wav' [(0.071375154, 137), (0.018485669, 0), (0.013933207, 72), (0.007914148, 87), (0.005302118, 88)]
b'dane_recording_19_42_37_20190208.pcm_out00141.wav' [(0.11308347, 0), (0.023913613, 137), (0.018357249, 72), (0.01530184, 5